In [14]:
import sys
sys.path.append('./model_construction/')

In [15]:

import logging
import time
from argparse import ArgumentParser

import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

import datasets
import utils
import model
from nni.nas.pytorch.utils import AverageMeter
from nni.retiarii import fixed_arch
import random
model.add_pruning = True


In [16]:
logger = logging.getLogger('nni')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter()


In [17]:
import json
with open('./model_construction/darts_fmnist2/checkpoint.json') as inp:
    base = json.loads(inp.read())

In [28]:
!mkdir model_construction/random_pruned

In [27]:
base= {"normal_n2_p0": "zero", "normal_n2_p1": "zero", "normal_n3_p0": "zero", "normal_n3_p1": "maxpool", "normal_n3_p2": "zero", "normal_n4_p0": "zero", "normal_n4_p1": "sepconv3x3", "normal_n4_p2": "zero", "normal_n4_p3": "zero", "normal_n5_p0": "zero", "normal_n5_p1": "dilconv5x5", "normal_n5_p2": "zero", "normal_n5_p3": "zero", "normal_n5_p4": "zero", "reduce_n2_p0": "zero", "reduce_n2_p1": "maxpool", "reduce_n3_p0": "zero", "reduce_n3_p1": "zero", "reduce_n3_p2": "zero", "reduce_n4_p0": "zero", "reduce_n4_p1": "skipconnect", "reduce_n4_p2": "zero", "reduce_n4_p3": "sepconv5x5", "reduce_n5_p0": "maxpool", "reduce_n5_p1": "zero", "reduce_n5_p2": "zero", "reduce_n5_p3": "zero", "reduce_n5_p4": "zero", "normal_n2_switch": [1, 0], "normal_n3_switch": [2, 1], "normal_n4_switch": [3, 2], "normal_n5_switch": [2, 3], "reduce_n2_switch": [1, 0], "reduce_n3_switch": [2, 1], "reduce_n4_switch": [3, 2], "reduce_n5_switch": [4, 3]}

In [28]:
with fixed_arch(base):
        cnn_model = model.CNN(28, 1, 16, 8, 3, auxiliary=True, node_cls=model.ZeroNode)

[2022-07-03 12:46:20] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'sepconv3x3', 'normal_n4_p2': 'zero', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'zero', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch': [3, 2], 'reduce_n5_switch': [4, 3]}


In [29]:
dataset_train, dataset_valid = datasets.get_dataset("fmnist3", cutout_length=0)

In [30]:
train_loader = torch.utils.data.DataLoader(dataset_train,
                                               batch_size=4,
                                               shuffle=True,
                                               num_workers=1)

In [31]:
for x,y in train_loader:
    break

In [32]:
out = cnn_model(x)
abs(out[0][1] - out[0][0]).sum()

tensor(0.4911, grad_fn=<SumBackward0>)

In [24]:
out

(tensor([[ 0.5922,  0.2125, -0.3809,  0.1357, -0.0664, -0.4870, -0.2344,  0.3126],
         [ 0.3402, -0.0790, -0.1204,  0.2936,  0.1310, -0.2550, -0.4568, -0.0078],
         [ 0.3307,  0.0136, -0.3161,  0.1701,  0.3694, -0.1611, -0.3040,  0.2966],
         [ 0.3000,  0.0537, -0.2543,  0.2561,  0.3897, -0.1573, -0.3297,  0.3056]],
        grad_fn=<AddmmBackward0>),
 tensor([[-0.4580, -0.1015, -0.6316, -0.0802,  0.3517,  0.0903, -0.9294,  1.0053],
         [ 0.2011, -0.0562, -0.0564,  0.1440, -0.2666,  0.4330,  0.0825,  0.0837],
         [ 0.0804, -0.2507,  0.1191,  0.0589, -0.0267, -0.2484, -0.1389, -0.3647],
         [ 0.5067, -0.0649,  0.4189,  0.2502,  0.0966, -0.1374, -0.1802,  0.3715]],
        grad_fn=<AddmmBackward0>))

In [26]:
for run_num in range(3):
    for percent in [25, 50, 75]:
        while True:
            modified = base.copy()
            all_keys = [k for k in modified if 'switch' not in k]
            random.shuffle(all_keys)
            all_keys = all_keys[:len(all_keys)*percent//100]
            for k in all_keys:
                modified[k] = 'zero'
            with fixed_arch(modified):
                cnn_model = model.CNN(28, 1, 16, 8, 3, auxiliary=True, node_cls=model.ZeroNode)
            out = cnn_model(x)
            if abs(out[0][1] - out[0][0]).sum()>1e-3:
                print (abs(out[0][1] - out[0][0]).sum())
                with open(f'model_construction/random_pruned/{run_num}_{percent}.json', 'w') as out:
                    out.write(json.dumps(modified))
                    
                    break
                
            else:
                print ('zero path', run_num, percent)

[2022-07-03 12:45:49] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'zero', 'normal_n4_p3': 'sepconv5x5', 'normal_n5_p0': 'dilconv3x3', 'normal_n5_p1': 'dilconv5x5', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'dilconv5x5', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'maxpool', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'skipconnect', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'sepconv5x5', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'dilconv5x5', 'reduce_n5_p2': 'dilconv5x5', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'sepconv5x5', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 

zero path 2 50
[2022-07-03 12:45:50] Fixed architecture: {'normal_n2_p0': 'sepconv3x3', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'sepconv3x3', 'normal_n3_p1': 'maxpool', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'sepconv5x5', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'maxpool', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'sepconv3x3', 'normal_n5_p3': 'dilconv5x5', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'dilconv5x5', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'dilconv3x3', 'reduce_n3_p1': 'sepconv5x5', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'sepconv5x5', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'sepconv5x5', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'maxpool', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'sepconv3x3', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch': [3, 2],

In [40]:
for run_num in range(3):
    for percent in [100]:
        while True:
            modified = base.copy()
            all_keys = [k for k in modified if 'switch' not in k]
            random.shuffle(all_keys)
            all_keys = all_keys[:len(all_keys)*percent//100]
            for k in all_keys:
                modified[k] = 'zero'
            with fixed_arch(modified):
                cnn_model = model.CNN(28, 1, 16, 8, 3, auxiliary=True)
            out = cnn_model(x)
            if abs(out[0][1] - out[0][0]).sum()>1e-3:
                with open(f'model_construction/random_pruned/{run_num}_{percent}.json', 'w') as out:
                    out.write(json.dumps(modified))
                    break
            else:
                print ('zero path')

[2022-07-02 00:04:05] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'zero', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'zero', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch': [3, 2], 'reduce_n5_switch': [4, 3]}
zero path
[2022-07-02 00:04:05] Fixed architecture: {'normal_

zero path
[2022-07-02 00:04:07] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'zero', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'zero', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch': [3, 2], 'reduce_n5_switch': [4, 3]}
zero path
[2022-07-02 00:04:07] Fixed architecture:

zero path
[2022-07-02 00:04:08] Fixed architecture: {'normal_n2_p0': 'zero', 'normal_n2_p1': 'zero', 'normal_n3_p0': 'zero', 'normal_n3_p1': 'zero', 'normal_n3_p2': 'zero', 'normal_n4_p0': 'zero', 'normal_n4_p1': 'zero', 'normal_n4_p2': 'zero', 'normal_n4_p3': 'zero', 'normal_n5_p0': 'zero', 'normal_n5_p1': 'zero', 'normal_n5_p2': 'zero', 'normal_n5_p3': 'zero', 'normal_n5_p4': 'zero', 'reduce_n2_p0': 'zero', 'reduce_n2_p1': 'zero', 'reduce_n3_p0': 'zero', 'reduce_n3_p1': 'zero', 'reduce_n3_p2': 'zero', 'reduce_n4_p0': 'zero', 'reduce_n4_p1': 'zero', 'reduce_n4_p2': 'zero', 'reduce_n4_p3': 'zero', 'reduce_n5_p0': 'zero', 'reduce_n5_p1': 'zero', 'reduce_n5_p2': 'zero', 'reduce_n5_p3': 'zero', 'reduce_n5_p4': 'zero', 'normal_n2_switch': [1, 0], 'normal_n3_switch': [2, 1], 'normal_n4_switch': [3, 2], 'normal_n5_switch': [2, 3], 'reduce_n2_switch': [1, 0], 'reduce_n3_switch': [2, 1], 'reduce_n4_switch': [3, 2], 'reduce_n5_switch': [4, 3]}
zero path
[2022-07-02 00:04:08] Fixed architecture:

KeyboardInterrupt: 

In [37]:
out[1]

tensor([[-0.0206,  0.0083,  0.0284, -0.0095, -0.0003, -0.0194, -0.0172,  0.0078],
        [-0.0206,  0.0083,  0.0284, -0.0095, -0.0003, -0.0194, -0.0172,  0.0078],
        [-0.0206,  0.0083,  0.0284, -0.0095, -0.0003, -0.0194, -0.0172,  0.0078],
        [-0.0206,  0.0083,  0.0284, -0.0095, -0.0003, -0.0194, -0.0172,  0.0078]],
       grad_fn=<AddmmBackward0>)

tensor([0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<SubBackward0>)